In [1]:
# import modules
import os
import shutil
from datetime import date

In [4]:
'''Copying and renaming files'''

def bidsify(origpath, destpath, n_sessions=2, scan_types=None, detect_size=True, log_changes=True, log_name='CHANGES', verbose=False):
    
    
    # NOTE: mapping old names to new names by passing a dict is not currently supported (scan_names kwarg).
    # detect_size must be True (default) to work. Passing custom file size ranges not currently supported (file_sizes kwarg)
    
    # convert to absolute paths if relative path is passed
    origpath_abs = os.path.abspath(origpath)
    destpath_abs = os.path.abspath(destpath)
    
    # instantiate list of unsuccessfully handled file paths
    problem_files = []
    
    # default output folder names
    if scan_types is None:
        scan_types = ['anat','func','log']
    
    # for name mapping based on file size
    if detect_size:    #and scan_names is None:
        method = 'size'
        
    else:
        raise ValueError('detect_size must be True. Mapping by other file attributes not currently supported.')
    
    # for name mapping based on passed names
#     elif not detect_size and scan_names is None:
#         # use default dictionary of scan types
#         scan_names = {
#             'mprage' : 'T1w',
#             'bold1' : 'task-rest_run-01_bold',
#             'bold2' : 'task-mcr_run-02_bold',
#             'bold3' : 'task-swm_run-03_bold',
#             'bold4' : 'task-dd_run-04_bold',
#             'bold5' : 'task-rest_run-05_bold'
#             }
#         method = 'name'
    
#     elif detect_size and scan_names:
#         raise ValueError('Cannot map by both scan_names and detect_size')
        
#     if scan_names and type(scan_names) is not dict:
#         raise TypeError('scan_names must be a dict')

    for i, (root, dirs, files) in enumerate(os.walk(origpath_abs)):
        # create new directory structure
        if i == 0:
            [os.makedirs(destpath_abs+'/'+direc.split('_')[0]+'/ses-'+str(ses+1)+'/'+scantype, exist_ok=True) for direc in dirs for scantype in scan_types for ses in range(n_sessions)]
        
        # move and rename files
        else:
            file_list = [f for f in files if not f.startswith('.')]
            if file_list:

                        
                if method == 'size':
                    new_filepaths, old_filepaths, p_fs = _rename_size(file_list, root, n_sessions, destpath_abs, log_changes, log_name, verbose)
                    
                    if old_filepaths and new_filepaths:
                        for old, new in zip(old_filepaths, new_filepaths):
                            shutil.copy(old, new)
                            
                            if verbose:
                                print('moved ' + old + ' to ' + new)
                            
                            if p_fs:
                                problem_files.extend(p_fs)
                                
#                 elif method == 'name':
#                     for file in file_list:
#                         old_filepath = os.path.join(root, file)
#                         new_filepath, problem_file = _rename_dict(file, root, n_sessions, scan_names, destpath_abs, log_changes, log_name, verbose)

#                         if new_filepath is not None:
#                             # move and rename
#                             shutil.copy(old_filepath, new_filepath)

#                             if verbose:
#                                 print('moved ' + old_filepath + ' to ' + new_filepath)

#                         else:
#                             problem_files.append(problem_file)       
                    
                    
                if problem_files:
                    print('The following files were not successfully converted: ' + problem_files)

In [5]:
'''Defines naming scheme for moved files'''

def _rename_dict(file, root, n_sessions, scan_names, destpath_abs, log_changes, log_name, verbose):

    # viarable to track unsuccessfully renamed files
    problem_file = None

    old_path = os.path.join(root, file)
    base, ext = os.path.splitext(file)
    splitpath = root.split('/')

    # get subject ID
    sub = splitpath[-2].split('_')[0]

    # get session number
    ses_number = splitpath[-2].split('_')[1]
    if int(ses_number) <= n_sessions:
        session = 'ses-'+ses_number

    else:
        print('unrecognized session number \'' + ses_number + '\' for subID ' + sub)
        problem_file = file

    # get scan type (or log)
    if splitpath[-1] == 'ANATOMY':
        runtype = 'anat'

    elif splitpath[-1] == 'FUNCTIONAL':
        runtype = 'func'

    elif splitpath[-1] == 'LOG':
        runtype = 'LOG'

    else:
        print('unrecognized scan or log folder ' + splitpath[-1] + ' for subID ' + sub)
        problem_file = file

        
    # format scan name (or preserve name of log file)
    if base in scan_names:
        new_name = scan_names[base]

    elif ext == '.log':
        new_name = base

    else:
        print('unrecognized scan name ' + base + ' for file ' + file)
        problem_file = file

    try:
        if runtype != 'LOG':
            new_path = os.path.join(destpath_abs, sub, session, runtype, sub+'_'+new_name+ext)
        else:
            new_path = os.path.join(destpath_abs, sub, session, runtype, new_name+ext)
            
        if log_changes:
            _writelog(log_name, old_path, destpath_abs, sub, session, new_name, ext, verbose)

    except NameError:
        new_path = None

    
    return new_path, problem_file

In [ ]:
def _rename_size(file_list, root, n_sessions, destpath_abs, log_changes, log_name, verbose):
    
    old_fps = [os.path.join(root,file) for file in file_list]
    new_fps = []
    splitpath = root.split('/')
    
    # get subject ID
    sub = splitpath[-2].split('_')[0]
    
    # deal with anatomical scans
    if splitpath[-1] == 'ANATOMY':
        runtype = 'anat'
        sizes_dict = {os.stat(file).st_size: file for file in old_fps}
        right_size = 28836192
        best_scan = sizes_dict[right_size] if right_size in sizes_dict else sizes_dict[min(sizes_dict.keys(), key=lambda k: abs(k-right_size))]
        old_file
        
        
    # deal with functional scans
    elif splitpath[-1] == 'FUNCTIONAL':
        runtype = 'func'
        rests = []

        for file in old_fps:
            size = os.stat(file).st_size
            if size in range(110590352, 110594352):
                new_base = 'task-rest_run-01_bold'
            elif size in range(77412752, 77416752):
                new_base = 'task-mcr_run-02_bold'
            elif size in range(82942352, 82946352):
                new_base = 'task-swm_run-03_bold'
                
                
    elif splitpath[-1] == 'LOG':
        runtype = 'LOG'
        
    else:
        print('unrecognized scan or log folder ' + splitpath[-1] + ' for subID ' + sub)
        problem_file = file

In [4]:
'''Writes/updates log of moving file'''

def _writelog(log_name, old_path, destpath_abs, sub, session, new_name, ext, verbose):
    
    filename = os.path.join(destpath_abs, sub, session, 'LOG', log_name + '.log')
    
    if os.path.exists(filename):
        mode = 'a'
    
    else:
        mode = 'w'
        
    with open(filename, mode) as f:
        f.write(
            date.today().strftime('%Y-%m-%d') + '\n' +
            ' - ' + new_name+ext + ' moved from ' + old_path + '\n'
        )
    
    if verbose:
        print('wrote to log file ' + filename)

In [45]:
os.stat('/Users/paxtonfitzpatrick/Desktop/testorig/2018_1_asd/FUNCTIONAL/bold2.txt').st_size

15

In [14]:
# if os.path.isfile('/Users/paxtonfitzpatrick/Desktop/egg6_mapping_tet.egg'):
#     print('success')
# else:
#     raise FileNotFoundError('not there')
#     print(3+4)

In [77]:
for i, (root, dirs, files) in enumerate(os.walk('/Users/paxtonfitzpatrick/Desktop/testorig/')):
    file_list = [f for f in files if not f.startswith('.')]
    if file_list:
        print(file_list)
        print('_______')
        print(root)

    #     print(dirs)
    #     print('_______')
#         print(files)
#         print('_______')

['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/LOG
['bold3.txt', 'bold2.txt', 'bold1.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2022_1_asd/FUNCTIONAL
['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/LOG
['bold3.txt', 'bold2.txt', 'bold1.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2020_1_asd/FUNCTIONAL
['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/LOG
['bold3.txt', 'bold2.txt', 'bold1.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2019_1_asd/FUNCTIONAL
['mprage.txt']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2021_1_asd/ANATOMY
['2018_1_asd.log']
_______
/Users/paxtonfitzpatrick/Desktop/testorig/2021_1_